In [12]:
pip install wfdb wget tqdm biosppy imbalanced-learn seaborn

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip uninstall matplotlib --yes
!pip install matplotlib==3.1.3 

Found existing installation: matplotlib 3.1.3
Uninstalling matplotlib-3.1.3:
  Successfully uninstalled matplotlib-3.1.3
  Using cached matplotlib-3.1.3-cp39-cp39-macosx_11_0_arm64.whl


In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import wget
import zipfile
import wfdb as wf
import os
import pickle
import sys
import datetime
import cv2
from scipy import signal
from scipy.signal import find_peaks
from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [15]:
# config
debug = False;

In [19]:
module = os.path.abspath('./WESAD/')
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [17]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [6]:
# def generateImage(subject_ids, folder_name):
data = DataManager()
global_max = 0;
all_subject_data = np.array([]);
    # subject_ids = [2, 3]
for subject_id in subject_ids:
    if(debug): print("Processing subject id: ", subject_id);
    subject = data.load(subject_id)
    if(debug):
        print('All data:', subject)

        ecg_data = np.array(subject[0]['ECG'])
        
        if(debug): 
            print(len(ecg_data))
            

        ecg_data_T = ecg_data.T
        ecg_all_data = ecg_data_T[0]

        fs = 700
        fc = 150  # Cut-off frequency of the filter
        w = fc / (fs / 2) # Normalize the frequency
        b, a = signal.butter(5, w, 'low')
        ecg_filtered = signal.filtfilt(b, a, ecg_all_data)
        
        if(debug):
            plt.plot(ecg_all_data[0:1000]);
            plt.plot(ecg_filtered[0:1000]);
            plt.show();
            

    #     height = ecg_base_filtered.max()*0.75;
    #     if(debug):
    #         print("Max :", ecg_base_filtered.max(), "mean: ", ecg_base_filtered.mean(), "Height: ", height);

        peaks_data, _ = find_peaks(ecg_filtered, wlen=85, distance=275, prominence=.3)
        
        if(debug):
            plt.plot(peaks_data, ecg_filtered[0:2500][peaks_data], "xr");
            plt.plot(ecg_filtered[0:2500]);
            plt.legend(['distance']);
            plt.show();


    #     maxDis = 0;
    #     for i in range(len(peaks_base)-1):
    #         if(maxDis < peaks_base[i+1] - peaks_base[i]):
    #             maxDis = peaks_base[i+1] - peaks_base[i];
    #     if(global_max < maxDis):
    #         global_max = maxDis;
    #     print("peaks_base dis ", maxDis);
    #     maxDis = 0;
    #     for i in range(len(peaks_stress)-1):
    #         if(maxDis < peaks_stress[i+1] - peaks_stress[i]):
    #             maxDis = peaks_stress[i+1] - peaks_stress[i];
    #     if(global_max < maxDis):
    #         global_max = maxDis;
    #     print("peaks_stress dis ", maxDis);

        heart_beat_base = np.split(ecg_base_filtered, peaks_base)
        heart_beat_stress = np.split(ecg_stress_filtered, peaks_stress)

        for idx, idxval in enumerate(peaks_base):
            if idx != 0 & idx != (len(peaks_base) -1):
                heart_beat_base[idx] = (heart_beat_base[idx] - heart_beat_base[idx].min()) / heart_beat_base[idx].ptp()
                zerocount = 1200 - heart_beat_base[idx].size
                heart_beat_base[idx] = np.pad(heart_beat_base[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
                heart_beat_base[idx] = np.append(heart_beat_base[idx], 1.0) #Beaseline = 1
    #             plt.plot(heart_beat_base[idx]);
    #             plt.show();

        for idx, idxval in enumerate(peaks_stress):
            if idx != 0 & idx != (len(peaks_stress) -1):
                heart_beat_stress[idx] = (heart_beat_stress[idx] - heart_beat_stress[idx].min()) / heart_beat_stress[idx].ptp()
                zerocount = 1200 - heart_beat_stress[idx].size
                heart_beat_stress[idx] = np.pad(heart_beat_stress[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
                heart_beat_stress[idx] = np.append(heart_beat_stress[idx], 0.0) #Stress = 0
    #             plt.plot(heart_beat_stress[idx]);
    #             plt.show();

        heart_beat_base = np.delete(heart_beat_base, [0, len(heart_beat_base) - 1]);
        heart_beat_stress = np.delete(heart_beat_stress, [0, len(heart_beat_stress) -1])

        heart_beat_all = np.concatenate((heart_beat_base, heart_beat_stress), axis=0)

        subject_data = np.array(list(heart_beat_all[:]), dtype=float)
        if(all_subject_data.size == 0):
            all_subject_data = subject_data
        else:
            if(subject_data.size != 0):
                all_subject_data = np.concatenate((all_subject_data, subject_data), axis=0)

        print("New data shape", subject_data.shape, "Total Shape: ", all_subject_data.shape)

    # print("global_max dis ", global_max);
    df_final_data_X = pd.DataFrame(data=all_subject_data[:, :-1])
    df_final_data_Y = pd.DataFrame(data=all_subject_data[:,-1])

    smote = SMOTE(sampling_strategy='minority')
    X_sm, y_sm = smote.fit_resample(df_final_data_X, df_final_data_Y)
    y_sm.value_counts()
    
    os.makedirs('./Subjectwise_data/'+folder_name+'/Time_domain/Baseline');
    os.makedirs('./Subjectwise_data/'+folder_name+'/Time_domain/Stress');
    
    
    for index, row in X_sm.iterrows():
        fig = plt.figure(frameon=False)
        plt.plot(row,)
        plt.xticks([]), plt.yticks([])
        for spine in plt.gca().spines.values():
            spine.set_visible(False)          
        if(y_sm[0][index] == 1.0):
            filename = './Subjectwise_data/'+folder_name+'/Time_domain/Baseline/' + str(index)+'.png'
        else:
            filename = './Subjectwise_data/'+folder_name+'/Time_domain/Stress/' + str(index)+'.png'
        fig.savefig(filename)
        plt.close()
        print(index);

In [10]:
!rm -fr Subjectwise_data

In [11]:
test_subject = [2]
rest_subjects = [3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [20]:
generateImage(test_subject, 'test_subject')

In [21]:
generateImage(rest_subjects, 'rest_subjects')

In [ ]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
data = DataManager()
global_max = 0;
all_subject_data = np.array([]);
# subject_ids = [2, 3]
for subject_id in subject_ids:
    if(debug): print("Processing subject id: ", subject_id);
    subject = data.load(subject_id)
    
    subject_ecg = np.array(subject['ECG'])
    if(debug): 
        print(len(subject_ecg))
        
    subject_ecg_T = subject_ecg.T
    subject_ecg_data = subject_ecg_T[0]
        
    fs = 700
    fc = 150  # Cut-off frequency of the filter
    w = fc / (fs / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    subject_ecg_data_filtered = signal.filtfilt(b, a, subject_ecg_data)
    if(debug):
        plt.plot(subject_ecg_data[0:1000]);
        plt.plot(subject_ecg_data_filtered[0:1000]);
        plt.show();
        
    peaks_subject_ecg, _ = find_peaks(subject_ecg_data_filtered, wlen=85, distance=275, prominence=.3)
    if(debug):
        plt.plot(peaks_subject_ecg, subject_ecg_data_filtered[0:2500][peaks_subject_ecg], "xr");
        plt.plot(subject_ecg_data_filtered[0:2500]);
        plt.legend(['distance']);
        plt.show();
        
    heart_beat_subject_ecg = np.split(subject_ecg_data_filtered, peaks_subject_ecg)
    
    for idx, idxval in enumerate(peaks_subject_ecg):
        if idx != 0 & idx != (len(peaks_subject_ecg) -1):
            heart_beat_subject_ecg[idx] = (heart_beat_subject_ecg[idx] - heart_beat_subject_ecg[idx].min()) / heart_beat_subject_ecg[idx].ptp()
            zerocount = 1200 - heart_beat_subject_ecg[idx].size
            heart_beat_subject_ecg[idx] = np.pad(heart_beat_subject_ecg[idx], (0, zerocount), 'constant', constant_values=(0.0, 0.0))
#             plt.plot(heart_beat_subject_ecg[idx]);
#             plt.show();
            
    heart_beat_subject_ecg = np.delete(heart_beat_subject_ecg, [0, len(heart_beat_subject_ecg) - 1]);
    
    subject_data = np.array(list(heart_beat_subject_ecg[:]), dtype=float)
    if(all_subject_data.size == 0):
        all_subject_data = subject_data
    else:
        if(subject_data.size != 0):
            all_subject_data = np.concatenate((all_subject_data, subject_data), axis=0)
            
    print("New data shape", subject_data.shape, "Total Shape: ", all_subject_data.shape)